<a href="https://colab.research.google.com/github/heeyooon/TNT_2021_winter_team_3/blob/main/week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **월간 데이콘7: 컴퓨터 비전 학습 경진대회**
https://dacon.io/competitions/official/235626/data/


**주제** 컴퓨터 비전 기초 학습

**배경** 컴퓨터 비전은 가장 빠르게 발전하는 인공 지능 분야 중 하나로 학계, 산업계에서 많은 연구가 이루어지고 있습니다. 손글씨 이미지인 MNIST 데이터 세트는 이 분야의 고전적인 문제로 잘 알려져 있는데요, 이번 월간 데이콘 7에서는 MNIST 변형 데이터 세트를 제공드립니다. 컴퓨터 비전 기초를 공부하고 장학금도 받을 수 있는 기회를 놓치지 마세요!

**목표** 글자에 가려진 숫자 예측

**데이터** 일반적인 MNIST에 글자로 숫자가 가려진 데이터입니다.
1. train.csv : 학습에 필요한 데이터로 2048개의 이미지 데이터가 Flatten 되어있는 파일
2. test.csv : 검증 데이터 20480개입니다.


# **1. Data Load** 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split

from keras import layers
from keras import models

In [3]:
train = pd.read_csv('/content/drive/Shareddrives/TNT/스터디/2021년 겨울방학/과제/6주차 과제/data/train.csv', index_col = 0)
test = pd.read_csv('/content/drive/Shareddrives/TNT/스터디/2021년 겨울방학/과제/6주차 과제/data/test.csv', index_col = 0)
submission = pd.read_csv('/content/drive/Shareddrives/TNT/스터디/2021년 겨울방학/과제/6주차 과제/data/submission.csv', index_col = 0)

In [4]:
## 784로 Flatten 되어 있는 데이터를 28*28 로 reshape

X_train = np.array(train.iloc[:, 2:]).reshape(-1, 28, 28, 1).astype(np.float)
X_test = np.array(test.iloc[:, 1:]).reshape(-1, 28, 28, 1).astype(np.float)
y_train = to_categorical(train['digit'].values)

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)

X_train shape:  (2048, 28, 28, 1)
y_train shape:  (2048, 10)
X_test shape:  (20480, 28, 28, 1)


In [ ]:
## 0 ~ 255 로 퍼져있는 데이터를 0 ~ 1 사이로 rescaling
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
## train, test split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1)
print(X_train.shape[0])
print(X_val.shape[0])

1474
164


# **2. Base Model**

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)              

In [ ]:
BATCH_SIZE = 32
EPOCH = 100

history = model.fit(X_train, y_train, batch_size = BATCH_SIZE, 
                    epochs = EPOCH, validation_data = (X_val, y_val))

In [75]:
def get_sub_csv(pred):
  
  new_pred = np.argmax(pred, axis = 1)
  submission['digit'] = new_pred
  file_name = 'submission_' + datetime.now().strftime("%m%d_%H%M%S") + '.csv'
  submission.to_csv(file_name)

  print("SAVE FILE : " + file_name)

In [ ]:
test_pred = model.predict(X_test)
get_sub_csv(test_pred)

## submission score : 0.7352941176

# **3. Try image augmentation I**

**참고자료1** https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator?version=nightly

**참고자료2** https://www.tensorflow.org/tutorials/images/data_augmentation?hl=ko#%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EA%B0%95_3

**참고자료3** https://dacon.io/competitions/official/235626/codeshare/1551?page=3&dtype=recent&ptype=pub

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=0.1,
                                   zoom_range=0.1,
                                   fill_mode='constant',
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 128)             

In [ ]:
history = model.fit(train_datagen.flow(X_train, y_train, batch_size = 64),
                    epochs=100,
                    validation_data = test_datagen.flow(X_val, y_val)
                    )

In [ ]:
X_test = X_test / 225.
test_pred = model.predict(X_test)

get_sub_csv(test_pred)

# submission score : 0.0735294118

# **4. Try image augmentation II**

**참고자료**
https://dacon.io/competitions/official/235626/codeshare/1669?page=2&dtype=recent&ptype=pub

In [118]:
## train test split

from sklearn.model_selection import train_test_split

X_train = train.iloc[:, 1:]
y_train = train.iloc[:, 0]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 220)
X_train.head()

,letter,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96,M,3,1,4,0,4,0,4,4,2,1,2,2,0,2,2,0,0,0,1,1,2,1,1,0,0,3,4,4,0,4,1,0,0,2,2,4,4,3,3,...,0,0,0,2,0,3,0,4,3,3,0,1,2,4,2,1,1,2,3,2,1,1,2,1,1,1,2,3,0,0,4,4,1,3,0,1,2,1,2,2
1686,T,3,1,1,0,1,4,3,0,4,2,3,4,3,0,4,3,3,4,2,2,3,2,0,2,2,3,0,3,2,3,1,1,0,0,2,1,4,3,2,...,3,4,3,3,0,3,1,3,3,0,1,1,2,3,4,3,4,2,4,3,3,3,3,2,0,2,3,1,1,0,2,2,4,1,3,3,3,2,1,2
1660,C,2,3,3,2,0,2,3,3,4,2,0,4,4,3,2,1,4,0,0,1,2,2,0,4,0,4,3,2,3,4,4,1,2,1,1,2,2,0,4,...,3,1,4,1,3,1,1,0,3,3,4,1,3,3,1,1,0,1,1,3,0,4,2,3,4,0,3,4,4,2,3,3,2,4,2,3,0,1,2,4
1814,M,4,0,4,1,0,0,4,1,3,1,2,4,3,2,4,4,4,1,0,1,3,1,1,4,2,2,2,0,0,0,4,4,2,0,3,1,4,1,4,...,4,0,3,0,2,4,4,1,1,1,1,2,4,0,3,1,4,0,3,1,1,3,4,1,3,2,2,4,2,2,4,3,4,2,2,0,3,3,1,2
889,L,4,0,1,4,1,0,4,0,4,4,4,0,3,3,0,4,2,0,1,1,3,3,1,0,1,2,0,3,0,4,0,1,0,0,0,0,1,4,1,...,2,0,1,2,2,2,3,0,3,4,0,0,4,0,0,0,1,0,1,1,0,2,2,2,4,1,4,0,2,3,1,3,2,4,0,2,1,2,1,3


In [107]:
## conver to tensor 

# train set
X_train_tensor = tf.convert_to_tensor(X_train.iloc[:, 1:], dtype = tf.float32)
y_train_tensor = tf.squeeze(tf.convert_to_tensor(y_train, dtype = tf.int32))


# validation set
X_val_tensor = tf.convert_to_tensor(X_val.iloc[:, 1:], dtype = tf.float32)
y_val_tensor = tf.squeeze(tf.convert_to_tensor(y_val, dtype = tf.int32))

# test set
X_test_tensor = tf.convert_to_tensor(test.iloc[:, 1:], dtype = tf.float32)

In [108]:
import tensorflow as tf

def reshape_and_resize_train(tensor, label): 
  image_tensor = tf.reshape(tensor, (*IMAGE_SIZE, 1))
  image_tensor = tf.keras.layers.experimental.preprocessing.Resizing(*RESIZED_IMAGE_SIZE)(image_tensor)
  image_tensor = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(image_tensor)
    
  return image_tensor, label

def reshape_and_resize_test(tensor): 
  image_tensor = tf.reshape(tensor, (*IMAGE_SIZE, 1))
  image_tensor = tf.keras.layers.experimental.preprocessing.Resizing(*RESIZED_IMAGE_SIZE)(image_tensor)
  image_tensor = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(image_tensor)
  return image_tensor

###############################

IMAGE_SIZE = [28, 28]
RESIZED_IMAGE_SIZE = [227, 227]
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

train_ds = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor)
                            ).map(reshape_and_resize_train, num_parallel_calls = AUTO
                            ).cache(
                            ).shuffle(20000, reshuffle_each_iteration = True
                            ).batch(BATCH_SIZE)  

val_ds = tf.data.Dataset.from_tensor_slices((X_val_tensor, y_val_tensor)
                            ).map(reshape_and_resize_train, num_parallel_calls = AUTO
                            ).cache(
                            ).shuffle(20000, reshuffle_each_iteration = True
                            ).batch(BATCH_SIZE) 

test_ds = tf.data.Dataset.from_tensor_slices((X_test_tensor)
                            ).map(reshape_and_resize_test, num_parallel_calls = AUTO
                            ).cache(
                            ).batch(BATCH_SIZE)


In [113]:
DROPOUT_RATE = 0.2

## input
model_input = tf.keras.layers.Input((*RESIZED_IMAGE_SIZE, 1))

## preprocess :: image augmentation
x = tf.keras.layers.experimental.preprocessing.RandomZoom((-0.2, 0.1),(-0.2, 0.1), fill_mode = 'constant')(model_input)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1, fill_mode = 'constant')(x)

## AlexNet 참고

x = tf.keras.layers.Conv2D(96, kernel_size = 11, strides = 4, padding = 'valid', activation = 'relu')(x)
x = tf.keras.layers.MaxPooling2D((3, 3), strides = 2, padding = 'valid')(x)
x = tf.keras.layers.Conv2D(256, kernel_size = 5, strides = 1, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.MaxPooling2D((3, 3), strides = 2, padding = 'valid')(x)
x = tf.keras.layers.Conv2D(384, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Conv2D(384, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.Conv2D(256, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.MaxPooling2D((3, 3), strides = 2, padding = 'valid')(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4096, activation = 'relu')(x)
x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)

## output
model_output = tf.keras.layers.Dense(10, activation = 'softmax')(x)

model = tf.keras.Model(inputs = model_input, outputs = model_output)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])



model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 227, 227, 1)]     0         
_________________________________________________________________
random_zoom_6 (RandomZoom)   (None, 227, 227, 1)       0         
_________________________________________________________________
random_rotation_6 (RandomRot (None, 227, 227, 1)       0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 55, 55, 96)        11712     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 13, 13, 256)       0   

In [ ]:
from keras.callbacks import ModelCheckpoint

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
mc = ModelCheckpoint('0205_best_model.h5', monitor='val_accuracy', mode='min', save_best_only = True)

history = model.fit(train_ds, validation_data = val_ds, 
                    epochs = 100, callbacks = [early_stopping, mc])

In [117]:
loaded_model = tf.keras.models.load_model('0205_best_model.h5')

y_pred = loaded_model.predict(test_ds)
get_sub_csv(y_pred)

# epoch 100 >> submission score : 0.8774509804
# early stopping 51 >> submission score :  0.8039215686

SAVE FILE : submission_0205_081804.csv
